In [1]:
import pandas as pd
from fontTools.voltLib.ast import PositionAttachDefinition
from jupyter_server.services.api.handlers import APISpecHandler
from openpyxl.packaging.custom import StringProperty
from prompt_toolkit.layout.processors import BeforeInput
from tornado.locale import CSVLocale
#import OpenPyXL


In [2]:
df_stripe= pd.read_csv("C:/Users/ashra/PycharmProjects/Ashraf_Portfolio_Projects/data/raw/sales_stripe.csv")
display(df_stripe)

,txn_id,customer,email,amount,created_at,currency
0,tx1001,C001,john@example.com,100.0,2024-01-05,USD
1,tx1002,C002,mary@example.com,85.5,01/06/2024,usd
2,tx1003,NaN,alex@example.com,-20.0,2024/01/07,USD
3,tx1004,C004,sara@example.com,100.0,2024-13-01,USD
4,tx1002,C002,mary@example.com,85.5,01/06/2024,USD


Data Issues in sales_stripe.csv:

 txn_id: duplicate values present (tx1002)

 customer: leading/trailing spaces, missing values

 amount: negative values (-20), inconsistent numeric formatting

 created_at: multiple date formats, invalid date (2024-13-01)

 currency: inconsistent casing (usd, USD)

 Schema is acceptable but data quality is poor


In [3]:
df_api= pd.read_json("C:/Users/ashra/PycharmProjects/Ashraf_Portfolio_Projects/data/raw/sales_api.json")
display(df_api)

,id,customer_id,timestamp,amount_usd
0,api_9001,C005,2024-01-06 10:45:00+00:00,120
1,api_9002,,2024-01-07 12:00:00+00:00,75


Data Issues in sales_api.json:

missing customer ID

different field names

timestamp vs date

In [4]:
df_pos= pd.read_excel("C:/Users/ashra/PycharmProjects/Ashraf_Portfolio_Projects/data/raw/sales_pos.xlsx")
display(df_pos)

,order_id,cust_id,sale_date,total,currency
0,P5001,C001,2024-01-05,100.0,USD
1,P5002,C003,2024-01-06,90.0,usd
2,P5003,C004,2024-01-07,NaN,USD


Data Issues in sales_pos.xlsx:

null totals

different column names

inconsistent date formats

Schema:

transaction_id (string)  ---> txn_id, order_id, id

customer_id (string) ---> customer, customer_id,cust_id

date (date) ---> created_at, timestamp, sales_date

amount (float)  ---> amount, amount_usd, total

currency (string) ---> currency, currency

source (string) ---> what table it is coming from


In [5]:
CANONICAL_COLUMNS = [
    "transaction_id",
    "customer_id",
    "date",
    "amount",
    "currency",
    "source"
]


In [6]:
def clean_strings(df, columns):
    for col in columns:
        df[col] = df[col].astype(str).str.strip()
    return df


In [7]:
def clean_dates(df, column):
    df[column] = pd.to_datetime(
        df[column],
        errors="coerce"
    ).dt.date
    return df


In [8]:
def clean_amounts(df, column):
    df[column] = pd.to_numeric(df[column], errors="coerce")
    return df


In [9]:
def normalize_currency(df, column):
    df[column] = df[column].str.upper()
    return df


In [10]:
def drop_invalid_rows(df):

    df["customer_id"] = df["customer_id"].replace("", pd.NA)
    df = df.dropna(subset=["transaction_id", "customer_id", "date", "amount"])
    df = df[df["amount"] > 0]

    return df


In [11]:
def clean_stripe(df):
    df = df.rename(columns={
        "txn_id": "transaction_id",
        "customer": "customer_id",
        "created_at": "date"
    })

    df["source"] = "stripe"

    # clean string columns ONLY
    df["transaction_id"] = df["transaction_id"].astype(str).str.strip()
    df["customer_id"] = df["customer_id"].astype(str).str.strip()
    df["currency"] = df["currency"].astype(str).str.strip().str.upper()
    df["date"] = (df["date"].replace(["nan", "NaN", "None", ""], pd.NA))

    #

    # numeric + date parsing
    df["amount"] = pd.to_numeric(df["amount"], errors="coerce")
    #df["date"] = pd.to_datetime(df["date"],errors="coerce", dayfirst=False).dt.date
    df["date"] = pd.to_datetime(df["date"],errors="coerce",format="mixed").dt.date
    #df["date"] = pd.to_datetime(df["date"],errors="coerce").dt.date


    return df


In [12]:
#def clean_stripe(df):
#    df = df.rename(columns={
#        "txn_id": "transaction_id",
#        "customer": "customer_id",
#        "created_at": "date"
#    })
#    df["source"] = "stripe"
#
#    df = clean_strings(df, ["transaction_id", "customer_id", "currency"])
#    df = clean_dates(df, "date")
#    df = clean_amounts(df, "amount")
#    df = normalize_currency(df, "currency")
#
#    return df


In [13]:
df_stripe2 = clean_stripe(df_stripe)
display(df_stripe2)

,transaction_id,customer_id,email,amount,date,currency,source
0,tx1001,C001,john@example.com,100.0,2024-01-05,USD,stripe
1,tx1002,C002,mary@example.com,85.5,2024-01-06,USD,stripe
2,tx1003,nan,alex@example.com,-20.0,2024-01-07,USD,stripe
3,tx1004,C004,sara@example.com,100.0,NaT,USD,stripe
4,tx1002,C002,mary@example.com,85.5,2024-01-06,USD,stripe


In [14]:
def clean_pos(df):
    df = df.rename(columns={
        "order_id": "transaction_id",
        "cust_id": "customer_id",
        "sale_date": "date",
        "total": "amount"
    })
    df["source"] = "pos"

    df = clean_strings(df, ["transaction_id", "customer_id", "currency"])
    df["date"] = pd.to_datetime(df["date"], errors="coerce").dt.date
   #df = clean_dates(df, "date")
    df = clean_amounts(df, "amount")
    df = normalize_currency(df, "currency")

    return df


In [15]:
df_pos2 = clean_pos(df_pos)
display(df_pos2)

,transaction_id,customer_id,date,amount,currency,source
0,P5001,C001,2024-01-05,100.0,USD,pos
1,P5002,C003,2024-01-06,90.0,USD,pos
2,P5003,C004,2024-01-07,NaN,USD,pos


In [16]:
def clean_api(df):
    df = df.rename(columns={
        "id": "transaction_id",
        "timestamp": "date",
        "amount_usd": "amount"
    })
    df["currency"] = "USD"
    df["source"] = "api"

    df = clean_strings(df, ["transaction_id", "customer_id"])
    df["date"] = pd.to_datetime(df["date"], errors="coerce").dt.date

    #df = clean_dates(df, "date")
    df = clean_amounts(df, "amount")

    return df


In [17]:
df_api2 = clean_api(df_api)
display(df_api2)

,transaction_id,customer_id,date,amount,currency,source
0,api_9001,C005,2024-01-06,120,USD,api
1,api_9002,,2024-01-07,75,USD,api


In [18]:
df_stripe2[["transaction_id", "customer_id", "amount", "date"]]


,transaction_id,customer_id,amount,date
0,tx1001,C001,100.0,2024-01-05
1,tx1002,C002,85.5,2024-01-06
2,tx1003,nan,-20.0,2024-01-07
3,tx1004,C004,100.0,NaT
4,tx1002,C002,85.5,2024-01-06


In [19]:
df_all = pd.concat([df_stripe2, df_pos2, df_api2], ignore_index=True)
display(df_all)

,transaction_id,customer_id,email,amount,date,currency,source
0,tx1001,C001,john@example.com,100.0,2024-01-05,USD,stripe
1,tx1002,C002,mary@example.com,85.5,2024-01-06,USD,stripe
2,tx1003,nan,alex@example.com,-20.0,2024-01-07,USD,stripe
3,tx1004,C004,sara@example.com,100.0,NaT,USD,stripe
4,tx1002,C002,mary@example.com,85.5,2024-01-06,USD,stripe
5,P5001,C001,NaN,100.0,2024-01-05,USD,pos
6,P5002,C003,NaN,90.0,2024-01-06,USD,pos
7,P5003,C004,NaN,NaN,2024-01-07,USD,pos
8,api_9001,C005,NaN,120.0,2024-01-06,USD,api
9,api_9002,,NaN,75.0,2024-01-07,USD,api


In [20]:
df_all = df_all.drop_duplicates(subset=["transaction_id"])
display(df_all)

,transaction_id,customer_id,email,amount,date,currency,source
0,tx1001,C001,john@example.com,100.0,2024-01-05,USD,stripe
1,tx1002,C002,mary@example.com,85.5,2024-01-06,USD,stripe
2,tx1003,nan,alex@example.com,-20.0,2024-01-07,USD,stripe
3,tx1004,C004,sara@example.com,100.0,NaT,USD,stripe
5,P5001,C001,NaN,100.0,2024-01-05,USD,pos
6,P5002,C003,NaN,90.0,2024-01-06,USD,pos
7,P5003,C004,NaN,NaN,2024-01-07,USD,pos
8,api_9001,C005,NaN,120.0,2024-01-06,USD,api
9,api_9002,,NaN,75.0,2024-01-07,USD,api


In [21]:
df_all = df_all[CANONICAL_COLUMNS]
display(df_all)

,transaction_id,customer_id,date,amount,currency,source
0,tx1001,C001,2024-01-05,100.0,USD,stripe
1,tx1002,C002,2024-01-06,85.5,USD,stripe
2,tx1003,nan,2024-01-07,-20.0,USD,stripe
3,tx1004,C004,NaT,100.0,USD,stripe
5,P5001,C001,2024-01-05,100.0,USD,pos
6,P5002,C003,2024-01-06,90.0,USD,pos
7,P5003,C004,2024-01-07,NaN,USD,pos
8,api_9001,C005,2024-01-06,120.0,USD,api
9,api_9002,,2024-01-07,75.0,USD,api


In [22]:
df_all[df_all["transaction_id"] == "tx1002"]


,transaction_id,customer_id,date,amount,currency,source
1,tx1002,C002,2024-01-06,85.5,USD,stripe


In [23]:
df_all.loc[
    df_all["transaction_id"] == "tx1002",
    ["transaction_id", "customer_id", "date", "amount", "currency", "source"]
]


,transaction_id,customer_id,date,amount,currency,source
1,tx1002,C002,2024-01-06,85.5,USD,stripe


In [24]:
df_all = drop_invalid_rows(df_all)
display(df_all)

,transaction_id,customer_id,date,amount,currency,source
0,tx1001,C001,2024-01-05,100.0,USD,stripe
1,tx1002,C002,2024-01-06,85.5,USD,stripe
5,P5001,C001,2024-01-05,100.0,USD,pos
6,P5002,C003,2024-01-06,90.0,USD,pos
8,api_9001,C005,2024-01-06,120.0,USD,api


In [25]:
#df_all = pd.concat([df_stripe2, df_pos2, df_api2], ignore_index=True)
#df_all = drop_invalid_rows(df_all)
#df_all = df_all.drop_duplicates(subset=["transaction_id"])
#df_all = df_all[CANONICAL_COLUMNS]


In [27]:
assert df_all["transaction_id"].isna().sum() == 0
assert df_all["customer_id"].isna().sum() == 0
assert (df_all["amount"] <= 0).sum() == 0

In [29]:
display(df_all)

,transaction_id,customer_id,date,amount,currency,source
0,tx1001,C001,2024-01-05,100.0,USD,stripe
1,tx1002,C002,2024-01-06,85.5,USD,stripe
5,P5001,C001,2024-01-05,100.0,USD,pos
6,P5002,C003,2024-01-06,90.0,USD,pos
8,api_9001,C005,2024-01-06,120.0,USD,api


In [264]:
df_all.to_csv("C:/Users/ashra/PycharmProjects/Ashraf_Portfolio_Projects/data/final/clean_sales.csv", index=False)


In [196]:
#transaction_id,customer_id,date,amount,currency,source
#tx1001,C001,2024-01-05,100.00,USD,stripe
#tx1002,C002,2024-01-06,85.50,USD,stripe
#P5001,C001,2024-01-05,100.00,USD,pos
#P5002,C003,2024-01-06,90.00,USD,pos
#api_9001,C005,2024-01-06,120.00,USD,api
